<a href="https://colab.research.google.com/github/samuelvp360/SVP360CalculationController/blob/new_version/Activity_plots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install rdkit-pypi
!pip install -U matplotlib-inline
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 16.0 MB/s eta 0:00:00


In [4]:
%matplotlib inline

In [5]:
import matplotlib as mpl
import matplotlib.image as image
from matplotlib.offsetbox import (OffsetImage, AnnotationBbox)
from matplotlib import pyplot as plt
from matplotlib.figure import Figure
import numpy as np
import pandas as pd
import scipy.stats as stats
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from string import ascii_letters as letters
from pathlib import Path

In [57]:
df = pd.read_csv('Elicitation_results_05mM_no_out.csv', sep=',')
df = df.loc[:, [
    'Smiles',
    'Sample',
    'Treatment',
    'Daidzein',
    'Genistein',
    'Coumestrol',
    'Phaseollin'
]]
df = df.astype({
    'Phaseollin': 'float',
    'Daidzein': 'float',
    'Genistein': 'float',
    'Coumestrol': 'float'
})
df.dtypes

,0
Smiles,object
Sample,int64
Treatment,object
Daidzein,float64
Genistein,float64
Coumestrol,float64
Phaseollin,float64


In [58]:
df.set_index(['Treatment', 'Sample'], inplace=True)
df

Smiles  Daidzein  \
Treatment Sample                                                                
MeJas@0.5 2                             COC(=O)CC1CCC(=O)C1C\C=C/CC     0.530   
          3                             COC(=O)CC1CCC(=O)C1C\C=C/CC     0.260   
7a        2       COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)[C@@H...     3.821   
          3       COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)[C@@H...     1.783   
7b        1       C[C@@H](CC)[C@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)C(...     1.468   
          2       C[C@@H](CC)[C@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)C(...     1.135   
7c        1         COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)CC(C)C     0.911   
          3         COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)CC(C)C     3.433   
7d        2          COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)C(C)C     0.664   
          3          COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\C=C/CC)C(C)C     0.659   
7e        2               COC(=O)[C@H](C)NC(=O)CN1CCC(=O)N1C\C=C/CC     0.708   
          3               COC(=O)[C@H](C)NC(=O)CN1CCC(=O)N1C\C=C/CC     0.407   
7f        2       COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1CCCCC)[C@@H](C)CC     1.332   
          3       COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1CCCCC)[C@@H](C)CC     1.408   
7g        1       COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1CCCCCC)[C@@H](...     2.215   
          3       COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1CCCCCC)[C@@H](...     0.453   
7h        2         O=C(CN1CCC(=O)N1CCCC)N[C@H](C(=O)OC)[C@@H](C)CC     0.339   
          3         O=C(CN1CCC(=O)N1CCCC)N[C@H](C(=O)OC)[C@@H](C)CC     0.370   
7i        2       O=C(CCN1CCC(=O)N1C/C=C\CC)N[C@H](C(=O)OC)[C@@H...     0.484   
          3       O=C(CCN1CCC(=O)N1C/C=C\CC)N[C@H](C(=O)OC)[C@@H...     1.518   
(-)Blank  2                                                     NaN     0.524   
          3                                                     NaN     0.557   

                  Genistein  Coumestrol  Phaseollin  
Treatment Sample                                     
MeJas@0.5 2           0.174       2.216      25.999  
          3           0.336       1.762      21.200  
7a        2           0.441       6.140      48.978  
          3           0.194       7.295      44.712  
7b        1           0.506       9.516      70.155  
          2           0.310       8.027      55.648  
7c        1           0.117       5.261      37.455  
          3           0.375       4.745      37.761  
7d        2           0.227       4.070      36.634  
          3           0.147       4.433      38.382  
7e        2           0.188       4.115      39.127  
          3           0.107       5.108      52.885  
7f        2           0.322       8.535      42.548  
          3           0.330       7.568      39.578  
7g        1           0.429       6.889      23.175  
          3           0.258       5.684      19.897  
7h        2           0.100       2.508       7.249  
          3           0.072       2.527       7.470  
7i        2           0.101       0.863       8.657  
          3           0.182       1.130       5.860  
(-)Blank  2           0.161       2.546       7.980  
          3           0.183       2.271       5.345

In [59]:
pivot = pd.pivot_table(df, index='Treatment', values=[
    'Daidzein',
    'Genistein',
    'Coumestrol',
    'Phaseollin',
], aggfunc=np.mean, sort=False)
errors = pd.pivot_table(df, index='Treatment', values=[
    'Daidzein',
    'Genistein',
    'Coumestrol',
    'Phaseollin'
], aggfunc=np.std, sort=False)
pivot

<ipython-input-59-ff0913d17c86>:1: FutureWarning: The provided callable <function mean at 0x7bb008f17d90> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  pivot = pd.pivot_table(df, index='Treatment', values=[
<ipython-input-59-ff0913d17c86>:7: FutureWarning: The provided callable <function std at 0x7bb008f17eb0> is currently using DataFrameGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  errors = pd.pivot_table(df, index='Treatment', values=[


,Daidzein,Genistein,Coumestrol,Phaseollin
Treatment,,,,
MeJas@0.5,0.3950,0.2550,1.9890,23.5995
7a,2.8020,0.3175,6.7175,46.8450
7b,1.3015,0.4080,8.7715,62.9015
7c,2.1720,0.2460,5.0030,37.6080
7d,0.6615,0.1870,4.2515,37.5080
7e,0.5575,0.1475,4.6115,46.0060
7f,1.3700,0.3260,8.0515,41.0630
7g,1.3340,0.3435,6.2865,21.5360
7h,0.3545,0.0860,2.5175,7.3595


In [60]:
treatments = pivot.index.tolist()
metabolites = pivot.columns.tolist()
smi_list = df['Smiles'].dropna().unique().tolist()
smi_list

['COC(=O)CC1CCC(=O)C1C\\C=C/CC',
 'COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\\C=C/CC)[C@@H](C)CC',
 'C[C@@H](CC)[C@H](NC(=O)CN1CCC(=O)N1C\\C=C/CC)C(O)=O',
 'COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\\C=C/CC)CC(C)C',
 'COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1C\\C=C/CC)C(C)C',
 'COC(=O)[C@H](C)NC(=O)CN1CCC(=O)N1C\\C=C/CC',
 'COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1CCCCC)[C@@H](C)CC',
 'COC(=O)[C@@H](NC(=O)CN1CCC(=O)N1CCCCCC)[C@@H](C)CC',
 'O=C(CN1CCC(=O)N1CCCC)N[C@H](C(=O)OC)[C@@H](C)CC',
 'O=C(CCN1CCC(=O)N1C/C=C\\CC)N[C@H](C(=O)OC)[C@@H](C)CC']

In [61]:
mol_list = [Chem.MolFromSmiles(i) for i in smi_list]
mol_list

In [62]:
x = np.arange(len(treatments))  # the label locations
width = 0.2  # the width of the bars

In [63]:
responses = np.array([df.loc[t, m] for t in treatments for m in metabolites])
responses

array([[ 0.53 ,  0.26 ],
       [ 0.174,  0.336],
       [ 2.216,  1.762],
       [25.999, 21.2  ],
       [ 3.821,  1.783],
       [ 0.441,  0.194],
       [ 6.14 ,  7.295],
       [48.978, 44.712],
       [ 1.468,  1.135],
       [ 0.506,  0.31 ],
       [ 9.516,  8.027],
       [70.155, 55.648],
       [ 0.911,  3.433],
       [ 0.117,  0.375],
       [ 5.261,  4.745],
       [37.455, 37.761],
       [ 0.664,  0.659],
       [ 0.227,  0.147],
       [ 4.07 ,  4.433],
       [36.634, 38.382],
       [ 0.708,  0.407],
       [ 0.188,  0.107],
       [ 4.115,  5.108],
       [39.127, 52.885],
       [ 1.332,  1.408],
       [ 0.322,  0.33 ],
       [ 8.535,  7.568],
       [42.548, 39.578],
       [ 2.215,  0.453],
       [ 0.429,  0.258],
       [ 6.889,  5.684],
       [23.175, 19.897],
       [ 0.339,  0.37 ],
       [ 0.1  ,  0.072],
       [ 2.508,  2.527],
       [ 7.249,  7.47 ],
       [ 0.484,  1.518],
       [ 0.101,  0.182],
       [ 0.863,  1.13 ],
       [ 8.657,  5.86 ],


In [64]:
hsd_result = stats.tukey_hsd(*responses)

In [65]:
def diff_tukey(arr):
    '''
    arr:
        numpy range.
    return:
        a binary list. 1 if there are significative differences, 0 if not.
    '''
    return [i for i in map(lambda x: 1 if abs(x) < 0.05 else 0, arr)]

In [66]:
diff_array = [i for i in map(diff_tukey, hsd_result.pvalue)]
print(diff_array)

[[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 

In [67]:
diff_matrix = np.asmatrix(diff_array)
diff_matrix
np.savetxt('diff_matrix_05.csv', diff_matrix, delimiter=',')
diff_matrix_tril = np.tril(diff_matrix)

In [68]:
def sig_diff(diff_matrix):
    sig_diff_letters = []
    last = ''
    for i, row in enumerate(diff_matrix):
        if i == 0:
            sig_diff_letters.append(letters[i])
            last = letters[i]
#             print(f'row: {i}, last: {last}')
            continue
        this_row = []
        for j, col in enumerate(row):
            if j < i:
                to_add = sig_diff_letters[j]
                if col == 0 and len(to_add) == 1:
                    this_row.append(to_add)
                    #print(f'{i},{j}: {this_row}')

                else:
                    continue
            else:
                break
        if not this_row:
            idx_last_letter = letters.index(last[-1])
            last = letters[idx_last_letter + 1]
            this_row.append(last)
#             print(f'coord: {i}, {j}; this row: {this_row}')
        this_row = ''.join(sorted(set(this_row)))
        sig_diff_letters.append(this_row)
    return sig_diff_letters

In [69]:
sig_diff_letters = sig_diff(diff_matrix_tril)
sig_diff_letters

['a',
 'a',
 'a',
 'b',
 'a',
 'a',
 'a',
 'c',
 'a',
 'a',
 'a',
 'd',
 'a',
 'a',
 'a',
 'c',
 'a',
 'a',
 'a',
 'c',
 'a',
 'a',
 'a',
 'c',
 'a',
 'a',
 'a',
 'c',
 'a',
 'a',
 'a',
 'b',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a',
 'a']

In [70]:
%matplotlib auto

Using matplotlib backend: agg


In [71]:
mol_box_list = []
yerr_list = []
patterns = {
    'Daidzein': '/',
    'Genistein': 'x',
    'Coumestrol': '+',
    'Phaseollin': '\\'
}
#for i, mol in enumerate(mol_list):
#    Draw.MolToFile(mol, f'{i}.png')
#    mol_mpl = image.imread(f'{i}.png')
#    mol_box = OffsetImage(mol_mpl, zoom = 0.25)
#    mol_box_list.append(mol_box)
#    plt.imshow(mol_mpl)

multiplier = 0
x_positions = []
y_positions = []
fig, ax = plt.subplots(layout='constrained')
for attribute, measurement in pivot.items():
    offset = width * multiplier
    yerr = errors[attribute]
    yerr_list.extend(yerr)
    x_positions.append(x + offset)
    rects = ax.bar(
        x + offset,
        measurement,
        width,
        label=attribute,
        yerr=yerr,
        lw=1.5,
        edgecolor='black',
        capstyle='round',
        zorder=2,
        #hatch=patterns[attribute],
        error_kw=dict(
            ecolor='gray',
            elinewidth=1.5,
            capsize=5,
            capthick=1.5,
            zorder=1
        )
    )
    y_positions.append(measurement.tolist())

    multiplier += 1

y_pos_sorted = []
for i, _ in enumerate(treatments):
    for j in y_positions:
        y_pos_sorted.append(j[i])

y_offset = max(y_pos_sorted) * 0.2
y_pos_sorted = [y * 1.2 + 3 for y in y_pos_sorted]

plt.xticks(rotation=90)

x_positions = np.concatenate(x_positions)
x_positions = sorted(x_positions)
coords = [i for i in zip(x_positions, y_pos_sorted)]

#for i in x:
 #   if i > 9:
  #      continue
   # ab = AnnotationBbox(mol_box_list[i], (i + 0.5, 60), frameon=False)
    #ax.add_artist(ab)
    #plt.annotate(
    #    treatments[i].split('@')[0], (i + 0.5, 83),
    #    ha='center', va='center', size=30,
    #)

for letter, position in zip(sig_diff_letters, coords):
    plt.annotate(
        letter, position, ha='center', va='center',
        size=10
    )

ax.set_ylabel('\u03BCg/g f.w.', fontsize=15)
ax.set_xlabel('Treatment', fontsize=15)
# ax.set_title('Elicitation')
pos = x + 1.5 * width
ax.set_xticks(pos)
ax.set_xticklabels(treatments)
ax.tick_params(axis='both', which='major', labelsize=12, length=12)
ax.legend(loc='upper left', ncol=4, fontsize=15)

ax.set_ylim(0, 100)
fig.set_figwidth(12)
fig.set_figheight(9)
#plt.show()

In [72]:
plt.savefig('plot_05mM.svg')